# Creating the Tools with Standalone Callbacks 
Goal: create the tools so that people can use them as a python script

Steps to complete the main tool:
1. set up changeable channels with the most basic barebone example
2. link the two plots together
3. link to the frequency slider (mind the javacall back: callback_policy="mouseup")
4. set up histogram and bin changer
5. link histogram data. make sure it can be updated
6. fix minor details such as plot labels etc.

#### Barebone example: PSD plot with changeable channels

In [17]:
import sys
# locate your spectralCV so we have scv_funcs to use
sys.path.append('/Users/ldliao/Research/Projects/spectralCV/')
sys.path.append('/Users/ldliao/Research/Projects/spectralCV/notebooks_visualization/')

In [18]:
import os
# the only lines you need to change to inspect different saved data
path = '/Users/ldliao/Research/Projects/spectralCV/results/kjm_digits/bp/'
os.chdir(path)

In [19]:
# imports
import numpy as np
import scipy as sp
from scipy.stats import expon
import glob

import neurodsp as ndsp
from scv_funcs import lfpca
import warnings
warnings.filterwarnings('ignore')

In [21]:
# bokeh imports
import bokeh
from bokeh.io import push_notebook, show, output_notebook, curdoc
from bokeh.io import *
from bokeh.plotting import figure, output_file, show
from bokeh.layouts import gridplot, row, column
from bokeh.models import ColumnDataSource, TapTool, Slider, Span, CustomJS
from bokeh.models.widgets import PreText, Select, Slider
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

output_notebook()

Loading BokehJS ...

In [6]:
import os
# the only lines you need to change to inspect different saved data
path = '/Users/ldliao/Research/Projects/spectralCV/results/kjm_digits/bp/'
os.chdir(path)

In [22]:
# collect all the names of .npz files in the folder
lfpca_all_names = glob.glob("*.npz")
lfpca_all_names.sort()

# loading the npz files 
lfpca_all = {}
for ind, lf in enumerate(lfpca_all_names):
    lfpca_all[lf[:-4]] = lfpca.lfpca_load_spec(lf)

# initialize with the first lfpca object
lf = lfpca_all[lfpca_all_names[0][:-4]]

# grabbing channel count from psd
chan_count, freq = lf.psd.shape

# mapping all the channels
DEFAULT_TICKERS = list(map(str, range(chan_count)))
LF_TICKERS = [key for key in lfpca_all.keys()]

In [28]:
# setting up the defaults
chan = 0
select_freq = 10
select_bin = 20
freq_vals = lf.f_axis[1:]
psd_vals = lf.psd[chan].T[1:]
scv_vals = lf.scv[chan].T[1:]

attr = ColumnDataSource(data=dict(chan=[chan]))
source = ColumnDataSource(data=dict(freq_vals=freq_vals, 
                                    psd_vals=psd_vals, 
                                    scv_vals=scv_vals))

In [29]:
ch_ticker = Select(value=str(chan), title='channel', options=DEFAULT_TICKERS)

In [30]:
psd_plot = figure(title='PSD', x_axis_type='log', y_axis_type='log')
psd_plot.legend.location = 'top_left'
psd_plot.xaxis.axis_label = 'Frequency (Hz)'
psd_plot.yaxis.axis_label = 'Power/Frequency (dB/Hz)'
psd_plot.grid.grid_line_alpha=0.3

In [31]:
# customize plot to psd
def create_psd_plot(psd_plot, source):
    psd_plot.line('freq_vals', 'psd_vals', source=source, color='navy')

In [38]:
lf.f_axis[1:]

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
       100., 101., 102., 103., 104., 105., 106., 107., 108., 109., 110.,
       111., 112., 113., 114., 115., 116., 117., 118., 119., 120., 121.,
       122., 123., 124., 125., 126., 127., 128., 129., 130., 131., 132.,
       133., 134., 135., 136., 137., 138., 139., 140., 141., 142., 143.,
       144., 145., 146., 147., 148., 149., 150., 15

In [32]:
create_psd_plot(psd_plot, source)

In [43]:
widgets = row(ch_ticker)
layout = column(widgets, psd_plot)

chan_callback = CustomJS(args=dict(attr=attr, source=source), code="""
    var attr_data = attr.data
    attr_data['chan'] = cb_obj.value
    var f = cb_obj.value
    
    var data = source.data
    data['freq_vals'] = lf.f_axis[1,]
    data['psd_vals'] = lf.psd.f[[1,]]
    data['psd_vals'] = lf.scv.f[[1,]]

    source.change.emit();
    attr.change.emit();
""")

ch_ticker.js_on_change('value', chan_callback)

# In the notebook, just pass the function that defines the app to show
show(layout)

In [9]:
# initializing values for frequency, psd, scv, histogram plot
attr = ColumnDataSource(data=dict(chan=[chan],
                                  select_freq=[select_freq]))
                                  # select_bin=[]))
source = ColumnDataSource(data=dict(freq_vals=freq_vals, 
                                    psd_vals=psd_vals, 
                                    scv_vals=scv_vals))

# creating a selector and slider
lf_ticker = Select(value=lfpca_all_names[0][:-4], title='lf_condition', options=LF_TICKERS)
ticker = Select(value=str(chan), title='channel', options=DEFAULT_TICKERS)
freq_slider = Slider(start=1, end=199, value=select_freq, step=1, title="Frequency", callback_policy="mouseup")
# bin_slider = Slider(start=10, end=55, value=select_bin, step=5, title="Number of bins", callback_policy="mouseup")

# create data and selection tools
# source = ColumnDataSource(data=dict(freq_vals=freq_vals, psd_vals=psd_vals, scv_vals=scv_vals))

TOOLS = "help" #tapTool work in progress

# setting up plots
psd_plot = figure(tools=TOOLS, title='PSD', x_axis_type='log', y_axis_type='log')
psd_plot.legend.location = 'top_left'
psd_plot.xaxis.axis_label = 'Frequency (Hz)'
psd_plot.yaxis.axis_label = 'Power/Frequency (dB/Hz)'
psd_plot.grid.grid_line_alpha=0.3

scv_plot = figure(tools=TOOLS, title='SCV', x_axis_type='log', y_axis_type='log')
scv_plot.legend.location='top_left'
scv_plot.xaxis.axis_label = 'Frequency (Hz)'
scv_plot.yaxis.axis_label = '(Unitless)'
scv_plot.grid.grid_line_alpha=0.3

# customize plot to psd
def create_psd_plot(psd_plot, source):
    psd_plot.line('freq_vals', 'psd_vals', source=source, color='navy')

# customize plot to psd
def create_scv_plot(scv_plot, source):
    fit_line = bokeh.models.glyphs.Line(x='freq_vals', y=1, line_width=5, line_alpha=0.5, line_color='darkgrey')
    scv_plot.add_glyph(source, fit_line)
    scv_plot.line('freq_vals', 'scv_vals', source=source, color='navy')


# initializing plots
create_psd_plot(psd_plot, source)
create_scv_plot(scv_plot, source)
vline_psd = Span(location=select_freq, dimension='height', line_color='red', line_dash='dashed', line_width=3)
vline_scv = Span(location=select_freq, dimension='height', line_color='red', line_dash='dashed', line_width=3)
psd_plot.add_layout(vline_psd)
scv_plot.add_layout(vline_scv)

all_plots = gridplot([[psd_plot, scv_plot]], plot_width=300, plot_height=300)

chan_callback = CustomJS(args=dict(attr=attr, source=source), code="""
    var attr_data = attr.data
    attr_data['chan'] = cb_obj.value
    var select_freq = attr_data['select_freq']
    
    var data = source.data
    data['freq_vals'] = lf.f_axis[1:]
    data['psd_vals'] = lf.psd[cb_obj.value].T[1:]
    data['psd_vals'] = lf.scv[cb_obj.value].T[1:]

    source.change.emit();
    attr.change.emit();
""")

# chan_callback = CustomJS(args=dict(source=source), code="""
#     var data = source.data;
#     var f = cb_obj.value
#     var x = data['x']
#     var y = data['y']
#     for (var i = 0; i < x.length; i++) {
#         y[i] = Math.pow(x[i], f)
#     }
#     source.change.emit();
# """)

ticker.js_on_change('value', cond_callback)


# set up connector spans
freq_slider.callback = CustomJS(args=dict(span1 = vline_psd,
                                          span2 = vline_scv,
                                          slider = freq_slider),
                                          code = """span1.location = slider.value; 
                                                    span2.location = slider.value;""")

# organize layout
widgets = row(lf_ticker, ticker)
sliders = row(freq_slider)#, bin_slider)
layout = column(widgets, sliders, all_plots)
# doc.add_root(layout)

# In the notebook, just pass the function that defines the app to show
show(layout, notebook_handle=True)
# curdoc().add_root(layout) - for .py file

NameError: name 'cond_callback' is not defined

In [10]:
x = [x*0.005 for x in range(0, 200)]
y = x

source = ColumnDataSource(data=dict(x=x, y=y))

plot = Figure(plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

callback = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var f = cb_obj.value
    var x = data['x']
    var y = data['y']
    for (var i = 0; i < x.length; i++) {
        y[i] = Math.pow(x[i], f)
    }
    source.change.emit();
""")

slider = Slider(start=0.1, end=4, value=1, step=.1, title="power")
slider.js_on_change('value', callback)

layout = column(slider, plot)

show(layout)

In [13]:
lf1

In [12]:
# collect all the names of .npz files in the folder
lfpca_all_names = glob.glob("*.npz")
lfpca_all_names.sort()

# loading the npz files 
lfpca_all = {}
for ind, lf in enumerate(lfpca_all_names):
    lfpca_all[lf[:-4]] = lfpca.lfpca_load_spec(lf)

# initialize with the first lfpca object
lf1 = lfpca_all[lfpca_all_names[0][:-4]]
lf2 = lfpca_all[lfpca_all_names[1][:-4]]

# grabbing channel count from psd
chan_count, freq = lf1.psd.shape

# mapping all the channels
CH_TICKERS = list(map(str, range(chan_count)))
LF_TICKERS = [key for key in lfpca_all.keys()]

def nix(val, lst):
    return [x for x in lst if x != val]

# initializing values for frequency, psd, scv, histogram plot
chan = 0
select_freq = 10
freq_vals_1 = lf1.f_axis[1:]
freq_vals_2 = lf2.f_axis[1:]
psd_vals_1 = lf1.psd[chan].T[1:]
psd_vals_2 = lf2.psd[chan].T[1:]
scv_vals_1 = lf1.scv[chan].T[1:] 
scv_vals_2 = lf2.scv[chan].T[1:]

# creating a selector and slider
lf_ticker_1 = Select(value=lfpca_all_names[0][:-4], title='condition_1', 
                     options=nix(lfpca_all_names[1][:-4], LF_TICKERS))
lf_ticker_2 = Select(value=lfpca_all_names[1][:-4], title='condition_2', 
                     options=nix(lfpca_all_names[0][:-4], LF_TICKERS))

ch_ticker = Select(value=str(chan), title='channel', options=CH_TICKERS)
freq_slider = Slider(start=1, end=199, value=select_freq, step=1, title="Frequency", callback_policy="mouseup")

# create data and selection tools
source_1 = ColumnDataSource(data=dict(freq_vals_1=freq_vals_1, psd_vals_1=psd_vals_1, scv_vals_1=scv_vals_1))
source_2 = ColumnDataSource(data=dict(freq_vals_2=freq_vals_2, psd_vals_2=psd_vals_2, scv_vals_2=scv_vals_2))

# TOOLS = "help" #tapTool work in progress

# setting up plots
psd_plot = figure(title='PSD', x_axis_type='log', y_axis_type='log')
psd_plot.legend.location = 'top_left'
psd_plot.xaxis.axis_label = 'Frequency (Hz)'
psd_plot.yaxis.axis_label = 'Power/Frequency (dB/Hz)'
psd_plot.grid.grid_line_alpha=0.3

scv_plot = figure(title='SCV', x_axis_type='log', y_axis_type='log')
scv_plot.legend.location='top_left'
scv_plot.xaxis.axis_label = 'Frequency (Hz)'
scv_plot.yaxis.axis_label = '(Unitless)'
scv_plot.grid.grid_line_alpha=0.3

# customize plot to psd
def create_psd_plot(psd_plot, source_1, source_2):
    psd_plot.line('freq_vals_1', 'psd_vals_1',  source=source_1, 
                  color='navy', muted_color='navy', muted_alpha=0.2, 
                  legend='cond_1')
    psd_plot.line('freq_vals_2', 'psd_vals_2',  source=source_2, 
                  color="green", muted_color='green', muted_alpha=0.2, legend='cond_2')
    psd_plot.legend.location = "bottom_left"
    psd_plot.legend.click_policy="mute"

# customize plot to psd
def create_scv_plot(scv_plot, source_1, source_2):
    fit_line = bokeh.models.glyphs.Line(x='freq_vals_1', y=1, line_width=5, line_alpha=0.5, line_color='darkgrey')
    scv_plot.add_glyph(source_1, fit_line)
    scv_plot.line('freq_vals_1', 'scv_vals_1',  source=source_1, 
                  line_color="navy", muted_alpha=0.2, legend='cond_1')
                  # legend=lf_ticker_1.value)
    scv_plot.line('freq_vals_2', 'scv_vals_2',  source=source_2, 
                  line_color="green", muted_alpha=0.2, legend='cond_2')
                  # legend=lf_ticker_2.value)
    scv_plot.legend.location = "bottom_left"
    scv_plot.legend.click_policy="mute"

# initializing plots
create_psd_plot(psd_plot, source_1, source_2)
create_scv_plot(scv_plot, source_1, source_2)
vline_psd = Span(location=select_freq, dimension='height', line_color='red', line_dash='dashed', line_width=3)
vline_scv = Span(location=select_freq, dimension='height', line_color='red', line_dash='dashed', line_width=3)
psd_plot.add_layout(vline_psd)
scv_plot.add_layout(vline_scv)

all_plots = gridplot([[psd_plot, scv_plot]], plot_width=450, plot_height=400)

# set up connector spans
freq_slider.callback = CustomJS(args=dict(span1 = vline_psd,
                                          span2 = vline_scv,
                                          slider = freq_slider),
                                          code = """span1.location = slider.value; 
                                                    span2.location = slider.value""")
#     def update(selected=None):
#         t1, t2 = lf_ticker_1.value, lf_ticker_2.value.value

#         data = get_data(t1, t2)
#         source.data = source.from_df(data[['t1', 't2', 't1_returns', 't2_returns']])
#         source_static.data = source.data

#         update_stats(data, t1, t2)

#         corr.title.text = '%s returns vs. %s returns' % (t1, t2)
#         ts1.title.text, ts2.title.text = t1, t2

callback = CustomJS(args=dict(source=source), code="""
    data = source.data;
    f = cb_obj.value;
    if (f == "_50_free") {
        data['x'] = data.x_50_free;
        data['y'] = data.y_50_free;
    } else if (f == "_100_free") {
        data['x'] = data.x_100_free;
        data['y'] = data.y_100_free;
    }
    source.change.emit();
""")

def update(selected=None):
    # get current slider values
    chan = int(ch_ticker.value)
    lf1 = lfpca_all[lf_ticker_1.value]
    lf2 = lfpca_all[lf_ticker_2.value]
#         select_freq = freq_slider.value

    # update data
    freq_vals_1 = lf1.f_axis[1:]
    freq_vals_2 = lf2.f_axis[1:]
    psd_vals_1 = lf1.psd[chan].T[1:]
    psd_vals_2 = lf2.psd[chan].T[1:]
    scv_vals_1 = lf1.scv[chan].T[1:] 
    scv_vals_2 = lf2.scv[chan].T[1:]
    data_1 = dict(freq_vals_1=freq_vals_1, psd_vals_1=psd_vals_1, scv_vals_1=scv_vals_1)
    data_2 = dict(freq_vals_2=freq_vals_2, psd_vals_2=psd_vals_2, scv_vals_2=scv_vals_2)

    # create a column data source for the plots to share
    source_1.data = data_1
    source_2.data = data_2


#     # whenever a widget changes, the changes are tracked and histogram always updated
#     for widget in [lf_ticker, ticker, freq_slider]:
#         widget.on_change('value', update)

def lf_ticker_1_change(attrname, old, new):
    lf_ticker_2.options = nix(new, LF_TICKERS)
    update()

def lf_ticker_2_change(attrname, old, new):
    lf_ticker_1.options = nix(new, LF_TICKERS)
    update()

def ch_ticker_change(attrname, old, new):
    update()

lf_ticker_1.on_change('value', lf_ticker_1_change)
lf_ticker_2.on_change('value', lf_ticker_2_change)
ch_ticker.on_change('value', ch_ticker_change)

# organize layout
widgets = row(lf_ticker_1, lf_ticker_2, ch_ticker)
sliders = row(freq_slider)
layout = column(widgets, sliders, all_plots)
# doc.add_root(layout)

# In the notebook, just pass the function that defines the app to show
show(layout, notebook_handle=True)
# curdoc().add_root(layout) - for .py file

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/server.html



In [19]:
from bokeh.io import show, output_file, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.layouts import column, widgetbox
from bokeh.models import CustomJS, Select

first_event = "_50_free"
second_event = "_100_free"

data_source = {
    'x': [8, 9],
    'y': [0, 0],
    'x_50_free': [7, 8],
    'y_50_free': [22.23, 24.34],
    'x_100_free': [6, 6],
    'y_100_free': [23.22, 25.12]
}    

source = ColumnDataSource(data=data_source)

callback = CustomJS(args=dict(source=source), code="""
    data = source.data;
    f = cb_obj.value;
    if (f == 1) {
        data['x'] = data.x_50_free;
        data['y'] = data.y_50_free;
    } else if (f == 3) {
        data['x'] = data.x_100_free;
        data['y'] = data.y_100_free;
    }
    source.change.emit();
""")

select = Select(title="Option:", value="1", options=["1",
               "2", "3"])
select.js_on_change('value', callback) 

plot = figure(plot_width=400, plot_height=400, x_axis_type='datetime')
plot.line('x', 'y', source=source)    
show(column(widgetbox(select),plot))

In [10]:
# collect all the names of .npz files in the folder
lfpca_all_names = glob.glob("*.npz")
lfpca_all_names.sort()

# loading the npz files 
lfpca_all = {}
for ind, lf in enumerate(lfpca_all_names):
    lfpca_all[lf[:-4]] = lfpca.lfpca_load_spec(lf)

# initialize with the first lfpca object
lf1 = lfpca_all[lfpca_all_names[0][:-4]]
lf2 = lfpca_all[lfpca_all_names[1][:-4]]

# grabbing channel count from psd
chan_count, freq = lf1.psd.shape

# mapping all the channels
CH_TICKERS = list(map(str, range(chan_count)))
LF_TICKERS = [key for key in lfpca_all.keys()]

def nix(val, lst):
    return [x for x in lst if x != val]

# initializing values for frequency, psd, scv, histogram plot
chan = 0
select_freq = 10
freq_vals_1 = lf1.f_axis[1:]
freq_vals_2 = lf2.f_axis[1:]
psd_vals_1 = lf1.psd[chan].T[1:]
psd_vals_2 = lf2.psd[chan].T[1:]
scv_vals_1 = lf1.scv[chan].T[1:] 
scv_vals_2 = lf2.scv[chan].T[1:]

# creating a selector and slider
lf_ticker_1 = Select(value=lfpca_all_names[0][:-4], title='condition_1', 
                     options=nix(lfpca_all_names[1][:-4], LF_TICKERS))
lf_ticker_2 = Select(value=lfpca_all_names[1][:-4], title='condition_2', 
                     options=nix(lfpca_all_names[0][:-4], LF_TICKERS))

ch_ticker = Select(value=str(chan), title='channel', options=CH_TICKERS)
freq_slider = Slider(start=1, end=199, value=select_freq, step=1, title="Frequency", callback_policy="mouseup")

# create data and selection tools
source_1 = ColumnDataSource(data=dict(freq_vals_1=freq_vals_1, psd_vals_1=psd_vals_1, scv_vals_1=scv_vals_1))
source_2 = ColumnDataSource(data=dict(freq_vals_2=freq_vals_2, psd_vals_2=psd_vals_2, scv_vals_2=scv_vals_2))

# TOOLS = "help" #tapTool work in progress

# setting up plots
psd_plot = figure(title='PSD', x_axis_type='log', y_axis_type='log')
psd_plot.legend.location = 'top_left'
psd_plot.xaxis.axis_label = 'Frequency (Hz)'
psd_plot.yaxis.axis_label = 'Power/Frequency (dB/Hz)'
psd_plot.grid.grid_line_alpha=0.3

scv_plot = figure(title='SCV', x_axis_type='log', y_axis_type='log')
scv_plot.legend.location='top_left'
scv_plot.xaxis.axis_label = 'Frequency (Hz)'
scv_plot.yaxis.axis_label = '(Unitless)'
scv_plot.grid.grid_line_alpha=0.3

# customize plot to psd
def create_psd_plot(psd_plot, source_1, source_2):
    psd_plot.line('freq_vals_1', 'psd_vals_1',  source=source_1, 
                  color='navy', muted_color='navy', muted_alpha=0.2, 
                  legend='cond_1')
    psd_plot.line('freq_vals_2', 'psd_vals_2',  source=source_2, 
                  color="green", muted_color='green', muted_alpha=0.2, legend='cond_2')
    psd_plot.legend.location = "bottom_left"
    psd_plot.legend.click_policy="mute"

# customize plot to psd
def create_scv_plot(scv_plot, source_1, source_2):
    fit_line = bokeh.models.glyphs.Line(x='freq_vals_1', y=1, line_width=5, line_alpha=0.5, line_color='darkgrey')
    scv_plot.add_glyph(source_1, fit_line)
    scv_plot.line('freq_vals_1', 'scv_vals_1',  source=source_1, 
                  line_color="navy", muted_alpha=0.2, legend='cond_1')
                  # legend=lf_ticker_1.value)
    scv_plot.line('freq_vals_2', 'scv_vals_2',  source=source_2, 
                  line_color="green", muted_alpha=0.2, legend='cond_2')
                  # legend=lf_ticker_2.value)
    scv_plot.legend.location = "bottom_left"
    scv_plot.legend.click_policy="mute"

# initializing plots
create_psd_plot(psd_plot, source_1, source_2)
create_scv_plot(scv_plot, source_1, source_2)
vline_psd = Span(location=select_freq, dimension='height', line_color='red', line_dash='dashed', line_width=3)
vline_scv = Span(location=select_freq, dimension='height', line_color='red', line_dash='dashed', line_width=3)
psd_plot.add_layout(vline_psd)
scv_plot.add_layout(vline_scv)

all_plots = gridplot([[psd_plot, scv_plot]], plot_width=450, plot_height=400)

# set up connector spans
freq_slider.callback = CustomJS(args=dict(span1 = vline_psd,
                                          span2 = vline_scv,
                                          slider = freq_slider),
                                          code = """span1.location = slider.value; 
                                                    span2.location = slider.value""")
#     def update(selected=None):
#         t1, t2 = lf_ticker_1.value, lf_ticker_2.value.value

#         data = get_data(t1, t2)
#         source.data = source.from_df(data[['t1', 't2', 't1_returns', 't2_returns']])
#         source_static.data = source.data

#         update_stats(data, t1, t2)

#         corr.title.text = '%s returns vs. %s returns' % (t1, t2)
#         ts1.title.text, ts2.title.text = t1, t2

callback = CustomJS(args=dict(source=source), code="""
    data = source.data;
    f = cb_obj.value;
    if (f == "_50_free") {
        data['x'] = data.x_50_free;
        data['y'] = data.y_50_free;
    } else if (f == "_100_free") {
        data['x'] = data.x_100_free;
        data['y'] = data.y_100_free;
    }
    source.change.emit();
""")

# def update(selected=None):
#     # get current slider values
#     chan = int(ch_ticker.value)
#     lf1 = lfpca_all[lf_ticker_1.value]
#     lf2 = lfpca_all[lf_ticker_2.value]
# #         select_freq = freq_slider.value

#     # update data
#     freq_vals_1 = lf1.f_axis[1:]
#     freq_vals_2 = lf2.f_axis[1:]
#     psd_vals_1 = lf1.psd[chan].T[1:]
#     psd_vals_2 = lf2.psd[chan].T[1:]
#     scv_vals_1 = lf1.scv[chan].T[1:] 
#     scv_vals_2 = lf2.scv[chan].T[1:]
#     data_1 = dict(freq_vals_1=freq_vals_1, psd_vals_1=psd_vals_1, scv_vals_1=scv_vals_1)
#     data_2 = dict(freq_vals_2=freq_vals_2, psd_vals_2=psd_vals_2, scv_vals_2=scv_vals_2)

#     # create a column data source for the plots to share
#     source_1.data = data_1
#     source_2.data = data_2


#     # whenever a widget changes, the changes are tracked and histogram always updated
#     for widget in [lf_ticker, ticker, freq_slider]:
#         widget.on_change('value', update)

def lf_ticker_1_change(attrname, old, new):
    lf_ticker_2.options = nix(new, LF_TICKERS)
    update()

def lf_ticker_2_change(attrname, old, new):
    lf_ticker_1.options = nix(new, LF_TICKERS)
    update()

def ch_ticker_change(attrname, old, new):
    update()

ch_callback = CustomJS(args=dict(source=source), code="""
    data = source.data;
    f = cb_obj.value;
    if (f == "_50_free") {
        data['x'] = data.x_50_free;
        data['y'] = data.y_50_free;
    } else if (f == "_100_free") {
        data['x'] = data.x_100_free;
        data['y'] = data.y_100_free;
    }
    source.change.emit();
""")

select = Select(title="Option:", value="default", options=["default",
                first_event, second_event])
select.js_on_change('value', callback) 
    
lf_ticker_1.on_change('value', lf_ticker_1_change)
lf_ticker_2.on_change('value', lf_ticker_2_change)
ch_ticker.on_change('value', ch_ticker_change)


# organize layout
widgets = row(lf_ticker_1, lf_ticker_2, ch_ticker)
sliders = row(freq_slider)
layout = column(widgets, sliders, all_plots)
# doc.add_root(layout)

# In the notebook, just pass the function that defines the app to show
show(layout, notebook_handle=True)
# curdoc().add_root(layout) - for .py file

NameError: name 'first_event' is not defined

In [14]:
sample = lfpca_all[lf[:-4]]

In [11]:

callback = CustomJS(args=dict(source=source), code="""
    data = source.data;
    f = cb_obj.value;
    if (f == "_50_free") {
        data['x'] = data.x_50_free;
        data['y'] = data.y_50_free;
    } else if (f == "_100_free") {
        data['x'] = data.x_100_free;
        data['y'] = data.y_100_free;
    }
    source.change.emit();
""")